In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define folder paths
input_folder = r"C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber"
output_folder = os.path.join(input_folder, "output")
plots_folder = os.path.join(output_folder, "plots")
albedo_folder = os.path.join(output_folder, "albedo")
albedo_plots_folder = os.path.join(plots_folder, "albedo")

# Ensure necessary folders exist
for folder in [output_folder, plots_folder, albedo_folder, albedo_plots_folder]:
    os.makedirs(folder, exist_ok=True)

# Utility Functions
def smooth_data(series, window_size=15):
    """Smooth data using rolling average with min_periods to handle edges."""
    return series.rolling(window=window_size, center=True, min_periods=1).mean()

def process_txt_to_csv(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    capture_data = False
    spectral_data = []
    for line in lines:
        if ">>>>>Begin Spectral Data<<<<<" in line:
            capture_data = True
            continue
        if capture_data and line.strip():
            cleaned_line = line.replace(",", ".").strip()
            spectral_data.append(cleaned_line.split())

    if spectral_data:
        df = pd.DataFrame(spectral_data, columns=["Wavelength", "Intensity"])
        df["Wavelength"] = pd.to_numeric(df["Wavelength"], errors="coerce")
        df["Intensity"] = pd.to_numeric(df["Intensity"], errors="coerce")
        df_filtered = df[(df["Wavelength"] >= 380) & (df["Wavelength"] < 751)]
        df_filtered["Intensity"] = df_filtered["Intensity"].clip(lower=0).fillna(0)
        df_filtered.to_csv(output_file, index=False, header=False)

def process_and_smooth_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=["Wavelength", "Intensity"])
    df["Wavelength"] = pd.to_numeric(df["Wavelength"], errors="coerce")
    df.dropna(subset=["Wavelength", "Intensity"], inplace=True)
    df_filtered = df[(df["Wavelength"] >= 380) & (df["Wavelength"] < 751)]
    df_filtered["Rounded_Wavelength"] = np.floor(df_filtered["Wavelength"]).astype(int)
    df_grouped = df_filtered.groupby("Rounded_Wavelength", as_index=False)["Intensity"].mean()
    df_grouped["Smoothed_Intensity"] = smooth_data(df_grouped["Intensity"], window_size=15)
    df_grouped.dropna(inplace=True)
    return df_grouped

def plot_spectral_data(df_grouped, file_name):
    """Plot and save spectral data."""
    plt.figure(figsize=(10, 6))
    plt.plot(
        df_grouped["Rounded_Wavelength"], df_grouped["Intensity"], 
        linestyle='-', marker='o', color='tab:blue', linewidth=1.5, markersize=2, 
        label="Original Intensity")
    plt.plot(
        df_grouped["Rounded_Wavelength"], df_grouped["Smoothed_Intensity"], 
         color='tab:red', linewidth=2, 
        label="Smoothed Intensity")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Intensity")
    plt.title(f"Spectral Data: {file_name}")
    plt.legend()
    plt.grid(True)
    plot_file_path = os.path.join(plots_folder, file_name.replace(".csv", ".png"))
    plt.savefig(plot_file_path)
    plt.close()
    print(f"Plot saved to: {plot_file_path}")

def calculate_albedo(sample_file, ref_file, output_file):
    sample_df = pd.read_csv(sample_file, header=None)
    ref_df = pd.read_csv(ref_file, header=None)
    result_df = pd.DataFrame()
    result_df["Wavelength"] = ref_df[0]
    result_df["Albedo"] = (sample_df[2] / ref_df[2]).clip(0, 1)
    result_df["Smoothed_Albedo"] = smooth_data(result_df["Albedo"], window_size=15)
    result_df.to_csv(output_file, index=False, header=False)
    print(f"Saved albedo file to: {output_file}")
    return result_df

def plot_albedo(df, file_name):
    """Plot and save albedo data."""
    plt.figure(figsize=(10, 6))
    plt.plot(df["Wavelength"], df["Albedo"], linestyle='-', marker='o', color='tab:blue', linewidth=1.8, markersize=2, label="Original Albedo")
    plt.plot(df["Wavelength"], df["Smoothed_Albedo"],  color='tab:red', linewidth=2.2, label="Smoothed Albedo")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Albedo")
    plt.title(f"Albedo Plot: {file_name}")
    plt.grid(True)
    plt.legend()
    plot_file_path = os.path.join(albedo_plots_folder, file_name.replace("_albedo.csv", "_albedo.png"))
    plt.savefig(plot_file_path)
    plt.close()
    print(f"Albedo plot saved to: {plot_file_path}")

# Main Workflow
if __name__ == "__main__":
    # Step 1: Convert .txt files to .csv
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            input_file = os.path.join(input_folder, file_name)
            output_file = os.path.join(output_folder, file_name.replace(".txt", ".csv"))
            process_txt_to_csv(input_file, output_file)

    # Step 2: Process, smooth, and plot .csv files
    ref_file = os.path.join(output_folder, "ref_led_4000K_fiber_processed.csv")
    for file_name in os.listdir(output_folder):
        if file_name.endswith(".csv") and not file_name.endswith("_processed.csv"):
            file_path = os.path.join(output_folder, file_name)
            df_grouped = process_and_smooth_csv(file_path)
            
            # Save processed file
            output_file = file_path.replace(".csv", "_processed.csv")
            df_grouped.to_csv(output_file, index=False, header=False)
            plot_spectral_data(df_grouped, file_name)
    
    # Step 3: Calculate and plot spectral albedo
    for file_name in os.listdir(output_folder):
        if file_name.endswith("_processed.csv") and "ref" not in file_name:
            sample_file = os.path.join(output_folder, file_name)
            albedo_file = os.path.join(albedo_folder, file_name.replace("_processed.csv", "_albedo.csv"))
            albedo_df = calculate_albedo(sample_file, ref_file, albedo_file)
            plot_albedo(albedo_df, file_name.replace("_processed.csv", "_albedo.csv"))
    
    print("All processing completed successfully!")


C:\Users\brahim\AppData\Local\Temp\ipykernel_12404\32591464.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Intensity"] = df_filtered["Intensity"].clip(lower=0).fillna(0)
C:\Users\brahim\AppData\Local\Temp\ipykernel_12404\32591464.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Intensity"] = df_filtered["Intensity"].clip(lower=0).fillna(0)
C:\Users\brahim\AppData\Local\Temp\ipykernel_12404\32591464.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber\output\plots\led_4000K_USB4H112691__1__11-26-55-549.png
Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber\output\plots\led_4000K_USB4H112691__2__11-27-08-548.png
Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber\output\plots\led_4000K_USB4H112691__3__11-27-39-546.png
Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber\output\plots\led_4000K_USB4H112691__4__11-27-47-545.png
Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate School/projet apparence spectrale/mesure_projet_blender/fiber\output\plots\led_4000K_USB4H112691__5__11-28-27-542.png
Plot saved to: C:/Users/brahim/OneDrive - Institut Optique Graduate Sc